In [1]:
!pip install earthengine-api



In [15]:
!pip install folium


In [19]:
!pip install earthengine-api folium


In [21]:
import ee
import folium

# Load an administrative boundaries dataset (e.g., FAO GAUL 2015)
regions = ee.FeatureCollection('FAO/GAUL/2015/level1')

# Filter the dataset for Lombardy (Italy)
lombardy = regions.filter(ee.Filter.And(
    ee.Filter.eq('ADM1_NAME', 'Lombardia'),  # Region name
    ee.Filter.eq('ADM0_NAME', 'Italy')       # Country name
))

# Load the SRTM Digital Elevation Data
srtm = ee.Image('USGS/SRTMGL1_003')

# Mask the SRTM data to the Lombardy region
lombardySRTM = srtm.clip(lombardy)

# Compute terrain attributes: slope and aspect
terrain = ee.Terrain.products(lombardySRTM)

# Extract elevation, slope, and aspect layers
elevation = terrain.select('elevation')
slope = terrain.select('slope')
aspect = terrain.select('aspect')

# Create a map centered on Lombardy
map_center = [45.4642, 9.1900]  # Coordinates of Milan, Lombardy
map_lombardy = folium.Map(location=map_center, zoom_start=7)

# Define visualization parameters
elevation_vis_params = {
    'min': 0,
    'max': 3000,
    'palette': ['blue', 'green', 'red']
}

# Add elevation layer to the map
map_lombardy.add_ee_layer(elevation, elevation_vis_params, 'Elevation')

# Display the map
map_lombardy


AttributeError: 'Map' object has no attribute 'add_ee_layer'

In [23]:
import folium

def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add the custom function to folium.Map class
folium.Map.add_ee_layer = add_ee_layer


In [25]:
# Load an administrative boundaries dataset (e.g., FAO GAUL 2015)
regions = ee.FeatureCollection('FAO/GAUL/2015/level1')

# Filter the dataset for Lombardy (Italy)
lombardy = regions.filter(ee.Filter.And(
    ee.Filter.eq('ADM1_NAME', 'Lombardia'),  # Region name
    ee.Filter.eq('ADM0_NAME', 'Italy')       # Country name
))


In [27]:
# Load the SRTM Digital Elevation Data
srtm = ee.Image('USGS/SRTMGL1_003')

# Mask the SRTM data to the Lombardy region
lombardySRTM = srtm.clip(lombardy)


In [30]:
# Compute terrain attributes: slope and aspect
terrain = ee.Terrain.products(lombardySRTM)

# Extract elevation, slope, and aspect layers
elevation = terrain.select('elevation')
slope = terrain.select('slope')
aspect = terrain.select('aspect')


In [87]:
# Create a map centered on Lombardy
map_center = [45.4642, 9.1900]  # Coordinates of Milan, Lombardy
map_lombardy = folium.Map(location=map_center, zoom_start=7)

# Define visualization parameters
elevation_vis_params = {
    'min': 0,
    'max': 4000,
    'palette': ['blue', 'green', 'yellow','red' ]
}

# Add elevation layer to the map
map_lombardy.add_ee_layer(elevation, elevation_vis_params, 'Elevation')

# Display the map
map_lombardy


In [40]:
!pip install earthengine-api folium pandas scikit-learn

import ee
import folium
import pandas as pd
from sklearn.preprocessing import StandardScaler


In [93]:
# Import necessary libraries 
!pip install earthengine-api folium pandas scikit-learn pyvinecopulib numpy

import ee
import folium
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from pyvinecopulib import Vinecop, RVineStructure, BicopFamily, FitControlsVinecop


In [69]:
# Define the region of interest (Lombardy, Italy)
regions = ee.FeatureCollection('FAO/GAUL/2015/level1')
lombardy = regions.filter(ee.Filter.And(
    ee.Filter.eq('ADM1_NAME', 'Lombardia'),
    ee.Filter.eq('ADM0_NAME', 'Italy')
))

# Define the time period (2000-2020)
start_date = '2000-01-01'
end_date = '2020-12-31'


In [71]:
# Function to calculate monthly mean
def monthly_mean(img):
    return img.reduceRegion(reducer=ee.Reducer.mean(), geometry=lombardy, scale=1000).toImage()

# Load and process CHIRPS rainfall data
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY') \
    .filterDate(start_date, end_date) \
    .map(lambda img: img.clip(lombardy)) \
    .select('precipitation') \
    .mean()  # Calculate mean over the specified period

# Load and process MODIS temperature data (LST_Day_1km)
modis_temp = ee.ImageCollection('MODIS/006/MOD11A2') \
    .filterDate(start_date, end_date) \
    .map(lambda img: img.clip(lombardy)) \
    .select('LST_Day_1km') \
    .mean().multiply(0.02).subtract(273.15)  # Convert to Celsius


In [73]:
# Load and process MODIS NDVI data (updated to MOD13A1 V6.1)
modis_ndvi = ee.ImageCollection('MODIS/061/MOD13A1') \
    .filterDate(start_date, end_date) \
    .map(lambda img: img.clip(lombardy)) \
    .select('NDVI') \
    .mean()


In [75]:
# Function to extract data from an image
def extract_data(image, scale=1000):
    mean_dict = image.reduceRegion(reducer=ee.Reducer.mean(), geometry=lombardy, scale=scale)
    return mean_dict.getInfo()

# Extract data for each variable
rainfall_data = extract_data(chirps)
temperature_data = extract_data(modis_temp)
ndvi_data = extract_data(modis_ndvi)

# Combine into a single DataFrame
data = pd.DataFrame([rainfall_data, temperature_data, ndvi_data], index=['Rainfall', 'Temperature', 'NDVI']).T


In [77]:
# Normalize the data
scaler = StandardScaler()
data_normalized = scaler.fit_transform(data)
data_normalized = pd.DataFrame(data_normalized, columns=data.columns)


In [95]:

# Fit the vine copula to the normalized data
structure = RVineStructure.simulate(len(data_normalized.columns))
vine = Vinecop(data_normalized.values, structure=structure, controls=FitControlsVinecop(family_set=[BicopFamily.gaussian]))

# Alternatively, fit a basic vine copula using default structure and controls
# vine = Vinecop(data_normalized.values, controls=FitControlsVinecop(family_set=[BicopFamily.gaussian]))



In [97]:
# Generate samples from the fitted vine copula
samples = vine.simulate(len(data))

# Convert samples to DataFrame
samples_df = pd.DataFrame(samples, columns=data.columns)

# Inverse transform samples to original scale
samples_original_scale = scaler.inverse_transform(samples_df)
samples_original_scale = pd.DataFrame(samples_original_scale, columns=data.columns)


In [101]:
# Calculate probabilities for the conditional event
prob_sNDVI_less_than_0_SPI_less_than_minus1_3 = np.mean((samples_original_scale['NDVI'] < 0) & (samples_original_scale['Rainfall'] <= -1.3))
prob_sNDVI_less_than_0_SPI_less_than_minus1_3_STI_greater_than_1_3 = np.mean((samples_original_scale['NDVI'] < 0) & (samples_original_scale['Rainfall'] <= -1.3) & (samples_original_scale['Temperature'] > 1.3))

denominator = (prob_sNDVI_less_than_0_SPI_less_than_minus1_3 - prob_sNDVI_less_than_0_SPI_less_than_minus1_3_STI_greater_than_1_3)

if denominator != 0:
    P_sNDVI_given_SPI_STI = (prob_sNDVI_less_than_0_SPI_less_than_minus1_3 - prob_sNDVI_less_than_0_SPI_less_than_minus1_3_STI_greater_than_1_3) / denominator
else:
    P_sNDVI_given_SPI_STI = np.nan  # or any other appropriate value

print("Probability of NDVI < 0 given Rainfall ≤ -1.3 and Temperature > 1.3: ", P_sNDVI_given_SPI_STI)


Probability of NDVI < 0 given Rainfall ≤ -1.3 and Temperature > 1.3:  nan
